<a href="https://colab.research.google.com/github/yuseongil034/yuseongil/blob/main/0718_yolo11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Google Colab용 YouTube YOLO 영상 처리기
# 이 코드를 하나의 셀에서 실행하세요

# 1. 필요한 패키지 설치
print("📦 패키지 설치 중...")
!pip install ultralytics yt-dlp opencv-python matplotlib -q

# 2. 라이브러리 임포트
import cv2
import yt_dlp
import tempfile
import os
import time
import matplotlib.pyplot as plt
import logging
from pathlib import Path
from typing import Optional, Tuple, List, Dict, Any
import json
from datetime import datetime

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Jupyter 환경 감지
try:
    from IPython.display import clear_output, display, HTML
    JUPYTER_ENV = True
except ImportError:
    JUPYTER_ENV = False
    def clear_output(wait=True):
        pass

class YOLOVideoProcessor:
    """
    YouTube 영상 처리기 with YOLOv11 객체 탐지
    """

    def __init__(self, model_path: str = "yolo11n.pt"):
        """
        처리기 초기화

        Args:
            model_path: YOLO 모델 경로
        """
        self.model_path = model_path
        self.model = None
        self.stats = {
            'total_frames_processed': 0,
            'total_detections': 0,
            'processing_time': 0,
            'detected_classes': set()
        }

    def load_model(self) -> bool:
        """YOLOv11 모델 로드"""
        try:
            from ultralytics import YOLO
            print(f"🤖 YOLOv11 모델 로드 중: {self.model_path}")
            self.model = YOLO(self.model_path)
            print(f"✅ 모델 로드 완료!")
            return True
        except Exception as e:
            print(f"❌ 모델 로드 실패: {e}")
            return False

    def get_video_info(self, youtube_url: str) -> Optional[Dict[str, Any]]:
        """YouTube 영상 정보 가져오기"""
        ydl_opts = {
            'quiet': True,
            'no_warnings': True,
        }

        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info = ydl.extract_info(youtube_url, download=False)
                return {
                    'title': info.get('title', '알 수 없음'),
                    'duration': info.get('duration', 0),
                    'uploader': info.get('uploader', '알 수 없음'),
                    'view_count': info.get('view_count', 0),
                    'upload_date': info.get('upload_date', '알 수 없음'),
                    'url': youtube_url
                }
        except Exception as e:
            print(f"❌ 영상 정보 가져오기 실패: {e}")
            return None

    def format_time(self, seconds: float) -> str:
        """초를 MM:SS 형식으로 변환"""
        minutes = int(seconds // 60)
        seconds = int(seconds % 60)
        return f"{minutes:02d}:{seconds:02d}"

    def process_video(self, youtube_url: str, output_filename: str = "yolo_output.mp4",
                     start_time: float = 0, duration: int = 10, skip_frames: int = 3) -> bool:
        """
        YouTube 영상을 YOLO 탐지로 처리

        Args:
            youtube_url: YouTube 영상 URL
            output_filename: 출력 파일명
            start_time: 시작 시간(초)
            duration: 처리할 길이(초)
            skip_frames: 건너뛸 프레임 수

        Returns:
            성공시 True, 실패시 False
        """
        if not self.model and not self.load_model():
            return False

        # 다운로드 설정
        ydl_opts = {
            'format': 'mp4/best[height<=720]',
            'outtmpl': '/tmp/temp_video.%(ext)s',
            'quiet': True,
        }

        temp_video_path = None
        cap = None
        out = None

        try:
            # 영상 다운로드
            print("⬇️ YouTube 영상 다운로드 중...")
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info = ydl.extract_info(youtube_url, download=True)
                temp_video_path = ydl.prepare_filename(info)
                print(f"🎥 제목: {info['title']}")
                print(f"📊 길이: {info.get('duration', '알 수 없음')}초")

            # 비디오 캡처 초기화
            cap = cv2.VideoCapture(temp_video_path)
            if not cap.isOpened():
                raise ValueError("영상 파일을 열 수 없습니다")

            # 영상 속성 가져오기
            fps = cap.get(cv2.CAP_PROP_FPS)
            frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            video_duration = total_frames / fps

            # 시간 범위 설정
            end_time = min(start_time + duration, video_duration)
            start_frame = int(start_time * fps)
            end_frame = int(end_time * fps)

            print(f"📹 영상 정보: {frame_width}x{frame_height}, {fps:.2f} FPS")
            print(f"🎯 처리 구간: {self.format_time(start_time)} ~ {self.format_time(end_time)}")

            # 시작 프레임으로 이동
            if start_frame > 0:
                cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

            # 비디오 라이터 초기화
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            output_fps = fps / skip_frames
            out = cv2.VideoWriter(output_filename, fourcc, output_fps, (frame_width, frame_height))

            # 프레임 처리
            frame_num = start_frame
            processed_frames = 0
            start_processing_time = time.time()

            print("🔄 YOLO 추론 시작...")

            while True:
                ret, frame = cap.read()
                if not ret or frame_num >= end_frame:
                    break

                # 프레임 건너뛰기
                if (frame_num - start_frame) % skip_frames == 0:
                    try:
                        # YOLO 추론
                        results = self.model.predict(
                            frame,
                            imgsz=640,
                            verbose=False,
                            show=False,
                            save=False
                        )[0]

                        # 결과 시각화
                        annotated_frame = results.plot()
                        out.write(annotated_frame)
                        processed_frames += 1

                        # 통계 업데이트
                        if results.boxes is not None:
                            detections = len(results.boxes)
                            self.stats['total_detections'] += detections

                            # 탐지된 클래스 추적
                            for box in results.boxes:
                                class_id = int(box.cls[0])
                                class_name = self.model.names[class_id]
                                self.stats['detected_classes'].add(class_name)

                        # 진행률 표시
                        if processed_frames % 20 == 0:
                            progress = ((frame_num - start_frame) / (end_frame - start_frame)) * 100
                            print(f"⏳ 진행률: {progress:.1f}% | 처리 프레임: {processed_frames}")

                    except Exception as e:
                        print(f"⚠️ 프레임 {frame_num} 처리 중 오류: {e}")
                        continue

                frame_num += 1

            # 최종 통계
            total_time = time.time() - start_processing_time
            self.stats['processing_time'] = total_time
            self.stats['total_frames_processed'] = processed_frames

            print(f"\n✅ YOLO 영상 추론 완료!")
            print(f"📊 처리 시간: {total_time:.2f}초")
            print(f"📊 처리된 프레임: {processed_frames}")
            print(f"📊 총 탐지 수: {self.stats['total_detections']}")
            print(f"📊 탐지된 클래스: {', '.join(sorted(self.stats['detected_classes']))}")
            print(f"💾 출력 파일: {output_filename}")

            # 파일 크기 확인
            if os.path.exists(output_filename):
                file_size = os.path.getsize(output_filename) / (1024*1024)
                print(f"📁 파일 크기: {file_size:.2f} MB")

            return True

        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            return False

        finally:
            # 리소스 정리
            if cap:
                cap.release()
            if out:
                out.release()
            if temp_video_path and os.path.exists(temp_video_path):
                try:
                    os.remove(temp_video_path)
                    print("🗑️ 임시 파일 정리 완료")
                except:
                    pass

# 빠른 실행을 위한 함수
def quick_process(youtube_url: str, start_time: float = 0, duration: int = 10):
    """빠른 처리를 위한 간단한 함수"""
    print("🚀 빠른 YOLO 영상 처리 시작!")
    processor = YOLOVideoProcessor("yolo11n.pt")
    return processor.process_video(
        youtube_url=youtube_url,
        start_time=start_time,
        duration=duration,
        skip_frames=3
    )

# 사용 예시
def example_usage():
    """사용 예시"""
    print("💡 사용 예시:")
    print("quick_process('https://youtube.com/watch?v=...', start_time=30, duration=10)")
    print("\n또는:")
    print("processor = YOLOVideoProcessor('yolo11n.pt')")
    print("processor.process_video('https://youtube.com/watch?v=...', start_time=0, duration=15)")

# 즉시 실행
print("🎉 YOLO 영상 처리기 준비 완료!")
print("=" * 50)
example_usage()
print("=" * 50)

# Google Colab에서 다운로드 기능 추가
def download_result(filename: str = "yolo_output.mp4"):
    """결과 파일 다운로드"""
    try:
        from google.colab import files
        if os.path.exists(filename):
            files.download(filename)
            print(f"📥 {filename} 다운로드 시작!")
        else:
            print(f"❌ {filename} 파일을 찾을 수 없습니다.")
    except ImportError:
        print("❌ Google Colab 환경이 아닙니다.")

print("📥 결과 다운로드: download_result('yolo_output.mp4')")

📦 패키지 설치 중...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.0 MB/s eta 0:00:00
🎉 YOLO 영상 처리기 준비 완료!
💡 사용 예시:
quick_process('https://youtube.com/watch?v=...', start_time=30, duration=10)

또는:
processor = YOLOVideoProcessor('yolo11n.pt')
processor.process_video('https://youtube.com/watch?v=...', start_time=0, duration=15)
📥 결과 다운로드: download_result('yolo_output.mp4')


In [ ]:
# 먼저 필요한 패키지들을 설치하세요
# !pip install ultralytics yt-dlp opencv-python matplotlib

import cv2
import yt_dlp
import tempfile
import os
import time
import matplotlib.pyplot as plt
import logging
from pathlib import Path
from typing import Optional, Tuple, List, Dict, Any
import argparse
import json
import sys
from datetime import datetime

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Jupyter 환경 감지
try:
    from IPython.display import clear_output
    JUPYTER_ENV = True
except ImportError:
    JUPYTER_ENV = False
    def clear_output(wait=True):
        pass

# 전역 모델 변수
model = None

class YOLOVideoProcessor:
    """
    향상된 YouTube 영상 처리기 with YOLOv11 객체 탐지
    """

    def __init__(self, model_path: str = "yolo11n.pt"):
        """
        처리기 초기화

        Args:
            model_path: YOLO 모델 경로
        """
        self.model_path = model_path
        self.model = None
        self.stats = {
            'total_frames_processed': 0,
            'total_detections': 0,
            'processing_time': 0,
            'detected_classes': set()
        }

    def load_model(self) -> bool:
        """YOLOv11 모델 로드"""
        try:
            from ultralytics import YOLO
            self.model = YOLO(self.model_path)
            logger.info(f"🤖 YOLOv11 모델 로드 완료: {self.model_path}")
            return True
        except Exception as e:
            logger.error(f"❌ YOLOv11 모델 로드 실패: {e}")
            logger.error("💡 ultralytics 설치 필요: pip install ultralytics>=8.0.0")
            return False

    def get_video_info(self, youtube_url: str) -> Optional[Dict[str, Any]]:
        """
        YouTube 영상 정보 가져오기

        Args:
            youtube_url: YouTube 영상 URL

        Returns:
            영상 정보 딕셔너리 또는 실패시 None
        """
        ydl_opts = {
            'quiet': True,
            'no_warnings': True,
        }

        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info = ydl.extract_info(youtube_url, download=False)
                return {
                    'title': info.get('title', '알 수 없음'),
                    'duration': info.get('duration', 0),
                    'uploader': info.get('uploader', '알 수 없음'),
                    'view_count': info.get('view_count', 0),
                    'upload_date': info.get('upload_date', '알 수 없음'),
                    'description': info.get('description', '')[:200] + '...' if info.get('description') else '',
                    'url': youtube_url
                }
        except Exception as e:
            logger.error(f"❌ 영상 정보 가져오기 실패: {e}")
            return None

    def validate_time_range(self, start_time: float, end_time: float, duration: float) -> Tuple[float, float]:
        """
        시간 범위 검증 및 조정

        Args:
            start_time: 시작 시간(초)
            end_time: 종료 시간(초)
            duration: 영상 길이(초)

        Returns:
            검증된 (start_time, end_time) 튜플
        """
        start_time = max(0, start_time)
        end_time = min(end_time, duration)

        if start_time >= end_time:
            logger.warning(f"잘못된 시간 범위: {start_time}-{end_time}, 마지막 10초 사용")
            end_time = duration
            start_time = max(0, duration - 10)

        return start_time, end_time

    def format_time(self, seconds: float) -> str:
        """초를 MM:SS 형식으로 변환"""
        minutes = int(seconds // 60)
        seconds = int(seconds % 60)
        return f"{minutes:02d}:{seconds:02d}"

    def process_video(self, youtube_url: str, output_filename: str = "yolo_output.mp4",
                     start_time: float = 0, end_time: Optional[float] = None,
                     max_duration: int = 10, skip_frames: int = 5) -> bool:
        """
        YouTube 영상을 YOLO 탐지로 처리

        Args:
            youtube_url: YouTube 영상 URL
            output_filename: 출력 파일명
            start_time: 시작 시간(초)
            end_time: 종료 시간(초) (None이면 자동)
            max_duration: 최대 처리 길이
            skip_frames: 건너뛸 프레임 수

        Returns:
            성공시 True, 실패시 False
        """
        if not self.model and not self.load_model():
            return False

        # 출력 디렉토리 생성
        output_path = Path(output_filename)
        output_path.parent.mkdir(parents=True, exist_ok=True)

        # 다운로드 설정
        ydl_opts = {
            'format': 'mp4/best[height<=720]',  # 품질 제한 증가
            'outtmpl': os.path.join(tempfile.gettempdir(), 'temp_video.%(ext)s'),
            'quiet': True,
        }

        temp_video_path = None
        cap = None
        out = None

        try:
            # 영상 다운로드
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                logger.info("⬇️ YouTube 영상 다운로드 중...")
                info = ydl.extract_info(youtube_url, download=True)
                temp_video_path = ydl.prepare_filename(info)
                logger.info(f"🎥 제목: {info['title']}")
                logger.info(f"📊 길이: {info.get('duration', '알 수 없음')}초")

            # 비디오 캡처 초기화
            cap = cv2.VideoCapture(temp_video_path)
            if not cap.isOpened():
                raise ValueError("영상 파일을 열 수 없습니다")

            # 영상 속성 가져오기
            fps = cap.get(cv2.CAP_PROP_FPS)
            frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            video_duration = total_frames / fps

            # 시간 범위 설정
            if end_time is None:
                end_time = min(start_time + max_duration, video_duration)

            start_time, end_time = self.validate_time_range(start_time, end_time, video_duration)

            # 프레임 번호로 변환
            start_frame = int(start_time * fps)
            end_frame = int(end_time * fps)
            process_duration = end_time - start_time

            logger.info(f"📹 영상 정보: {frame_width}x{frame_height}, {fps:.2f} FPS")
            logger.info(f"🎯 처리 구간: {self.format_time(start_time)} ~ {self.format_time(end_time)} ({process_duration:.1f}초)")

            # 시작 프레임으로 이동
            if start_frame > 0:
                cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

            # 비디오 라이터 초기화
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            output_fps = fps / skip_frames
            out = cv2.VideoWriter(output_filename, fourcc, output_fps, (frame_width, frame_height))

            if not out.isOpened():
                raise ValueError("출력 영상 파일을 생성할 수 없습니다")

            # 프레임 처리
            frame_num = start_frame
            processed_frames = 0
            start_processing_time = time.time()

            logger.info("🔄 YOLO 추론 시작...")

            while True:
                ret, frame = cap.read()
                if not ret or frame_num >= end_frame:
                    break

                # 성능을 위한 프레임 건너뛰기
                if (frame_num - start_frame) % skip_frames == 0:
                    try:
                        # YOLO 추론 (기본 신뢰도 0.25 사용)
                        results = self.model.predict(
                            frame,
                            imgsz=640,
                            verbose=False,
                            show=False,
                            save=False
                        )[0]

                        # 결과 시각화
                        annotated_frame = results.plot()
                        out.write(annotated_frame)
                        processed_frames += 1

                        # 통계 업데이트
                        if results.boxes is not None:
                            detections = len(results.boxes)
                            self.stats['total_detections'] += detections

                            # 탐지된 클래스 추적
                            for box in results.boxes:
                                class_id = int(box.cls[0])
                                class_name = self.model.names[class_id]
                                self.stats['detected_classes'].add(class_name)

                        # 진행률 표시
                        if processed_frames % 30 == 0:
                            progress = ((frame_num - start_frame) / (end_frame - start_frame)) * 100
                            current_time = frame_num / fps
                            elapsed_time = time.time() - start_processing_time

                            logger.info(f"⏳ 진행률: {progress:.1f}% | 시간: {self.format_time(current_time)} | "
                                      f"처리 프레임: {processed_frames} | 경과: {elapsed_time:.1f}초")

                    except Exception as e:
                        logger.warning(f"⚠️ 프레임 {frame_num} 처리 중 오류: {e}")
                        continue

                frame_num += 1

            # 최종 통계
            total_time = time.time() - start_processing_time
            self.stats['processing_time'] = total_time
            self.stats['total_frames_processed'] = processed_frames

            logger.info(f"✅ YOLO 영상 추론 완료!")
            logger.info(f"📊 처리 시간: {total_time:.2f}초")
            logger.info(f"📊 처리된 프레임: {processed_frames}")
            logger.info(f"📊 총 탐지 수: {self.stats['total_detections']}")
            logger.info(f"📊 탐지된 클래스: {', '.join(sorted(self.stats['detected_classes']))}")
            logger.info(f"💾 출력 파일: {output_filename}")

            # 파일 크기 확인
            if os.path.exists(output_filename):
                file_size = os.path.getsize(output_filename) / (1024*1024)
                logger.info(f"📁 파일 크기: {file_size:.2f} MB")

            return True

        except Exception as e:
            logger.error(f"❌ 오류 발생: {e}")
            return False

        finally:
            # 리소스 정리
            if cap:
                cap.release()
            if out:
                out.release()
            if temp_video_path and os.path.exists(temp_video_path):
                try:
                    os.remove(temp_video_path)
                    logger.info("🗑️ 임시 파일 정리 완료")
                except:
                    pass

    def save_stats(self, filename: str = "processing_stats.json"):
        """처리 통계를 파일에 저장"""
        stats_copy = self.stats.copy()
        stats_copy['detected_classes'] = list(stats_copy['detected_classes'])
        stats_copy['timestamp'] = datetime.now().isoformat()

        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(stats_copy, f, indent=2, ensure_ascii=False)

        logger.info(f"📊 통계가 {filename}에 저장되었습니다")


def get_available_models() -> Dict[str, str]:
    """사용 가능한 YOLOv11 모델 목록"""
    return {
        'yolo11n.pt': 'YOLOv11 Nano - 가장 빠르고 가벼운 모델',
        'yolo11s.pt': 'YOLOv11 Small - 속도와 정확도의 균형',
        'yolo11m.pt': 'YOLOv11 Medium - 중간 크기 모델',
        'yolo11l.pt': 'YOLOv11 Large - 높은 정확도',
        'yolo11x.pt': 'YOLOv11 Extra Large - 최고 정확도'
    }


def interactive_setup() -> Tuple[str, str, float, Optional[float]]:
    """대화형 설정 인터페이스"""
    print("\n🎥 YouTube YOLO 영상 처리기")
    print("=" * 50)

    # YouTube URL 입력
    youtube_url = input("📋 YouTube 영상 URL을 입력하세요: ").strip()
    if not youtube_url:
        raise ValueError("URL이 필요합니다")

    # 모델 선택
    models = get_available_models()
    print("\n🤖 사용 가능한 YOLOv11 모델:")
    for i, (model_name, description) in enumerate(models.items(), 1):
        print(f"{i}. {model_name} - {description}")

    while True:
        try:
            choice = input(f"\n모델을 선택하세요 (1-{len(models)}, 기본값: 1): ").strip()
            if not choice:
                choice = "1"
            choice = int(choice)
            if 1 <= choice <= len(models):
                model_path = list(models.keys())[choice-1]
                break
            else:
                print(f"❌ 1부터 {len(models)} 사이의 숫자를 입력하세요")
        except ValueError:
            print("❌ 올바른 숫자를 입력하세요")

    # 영상 정보 가져오기
    processor = YOLOVideoProcessor()
    video_info = processor.get_video_info(youtube_url)

    if not video_info:
        raise ValueError("영상 정보를 가져올 수 없습니다")

    duration = video_info['duration']
    print(f"\n📹 제목: {video_info['title']}")
    print(f"⏱ 길이: {duration}초 ({processor.format_time(duration)})")
    print(f"👤 업로더: {video_info['uploader']}")

    # 시간 범위 선택
    print(f"\n⏰ 처리할 구간을 선택하세요 (0~{duration}초)")
    start_time = float(input(f"🎬 시작 시간 (초 또는 분:초, 기본값: 0): ") or "0")

    # 시간 형식 처리 (MM:SS)
    if isinstance(start_time, str) and ':' in str(start_time):
        parts = str(start_time).split(':')
        if len(parts) == 2:
            start_time = int(parts[0]) * 60 + int(parts[1])

    end_input = input(f"🏁 종료 시간 ({start_time+1}초~{duration}초, 기본값: 자동): ").strip()
    end_time = None
    if end_input:
        if ':' in end_input:
            parts = end_input.split(':')
            if len(parts) == 2:
                end_time = int(parts[0]) * 60 + int(parts[1])
        else:
            end_time = float(end_input)

    return youtube_url, model_path, start_time, end_time


def main():
    """메인 함수 with 명령줄 인터페이스"""
    # Jupyter 환경 체크 및 인수 정리
    if JUPYTER_ENV or any('-f' in arg for arg in sys.argv):
        # Jupyter 환경에서는 인수를 초기화
        sys.argv = [sys.argv[0]]

    parser = argparse.ArgumentParser(description='YouTube 영상 처리기 with YOLOv11')
    parser.add_argument('--url', type=str, help='YouTube 영상 URL')
    parser.add_argument('--model', type=str, default='yolo11n.pt', help='YOLO 모델 경로')
    parser.add_argument('--start', type=float, default=0, help='시작 시간(초)')
    parser.add_argument('--end', type=float, help='종료 시간(초)')
    parser.add_argument('--duration', type=int, default=10, help='최대 처리 길이(초)')
    parser.add_argument('--output', type=str, default='yolo_output.mp4', help='출력 파일명')
    parser.add_argument('--skip-frames', type=int, default=5, help='건너뛸 프레임 수')
    parser.add_argument('--interactive', action='store_true', help='대화형 모드 사용')

    # Jupyter 환경에서도 알 수 없는 인수 허용
    parser.add_argument('-f', '--connection-file', help='Jupyter kernel connection file (ignored)')

    try:
        args = parser.parse_args()
    except SystemExit:
        # argparse 오류 발생 시 대화형 모드로 전환
        args = argparse.Namespace(
            url=None, model='yolo11n.pt', start=0, end=None,
            duration=10, output='yolo_output.mp4', skip_frames=5,
            interactive=True
        )

    try:
        if args.interactive or not args.url:
            # 대화형 모드
            youtube_url, model_path, start_time, end_time = interactive_setup()
            output_filename = input("💾 출력 파일명 (기본값: yolo_output.mp4): ") or "yolo_output.mp4"
            skip_frames = int(input("⏭ 건너뛸 프레임 수 (기본값: 5): ") or "5")
        else:
            # 명령줄 모드
            youtube_url = args.url
            model_path = args.model
            start_time = args.start
            end_time = args.end
            output_filename = args.output
            skip_frames = args.skip_frames

        # 출력 파일 확장자 확인
        if not output_filename.endswith('.mp4'):
            output_filename += '.mp4'

        # 영상 처리
        processor = YOLOVideoProcessor(model_path)
        success = processor.process_video(
            youtube_url=youtube_url,
            output_filename=output_filename,
            start_time=start_time,
            end_time=end_time,
            max_duration=args.duration,
            skip_frames=skip_frames
        )

        if success:
            # 통계 저장
            processor.save_stats()
            print("\n🎉 처리가 성공적으로 완료되었습니다!")
        else:
            print("\n❌ 처리 실패!")

    except KeyboardInterrupt:
        print("\n🚫 사용자에 의해 중단되었습니다")
    except Exception as e:
        logger.error(f"❌ 예상치 못한 오류: {e}")


# 빠른 실행을 위한 함수
def quick_process(youtube_url: str, start_time: float = 0, duration: int = 10):
    """빠른 처리를 위한 간단한 함수"""
    processor = YOLOVideoProcessor("yolo11n.pt")  # 가장 빠른 모델 사용
    return processor.process_video(
        youtube_url=youtube_url,
        start_time=start_time,
        max_duration=duration,
        skip_frames=3  # 더 빠른 처리를 위해 프레임 수 조정
    )


# Jupyter/Colab에서 직접 실행하기 위한 함수
def run_in_jupyter():
    """Jupyter/Colab에서 직접 실행"""
    try:
        youtube_url, model_path, start_time, end_time = interactive_setup()
        output_filename = input("💾 출력 파일명 (기본값: yolo_output.mp4): ") or "yolo_output.mp4"
        skip_frames = int(input("⏭ 건너뛸 프레임 수 (기본값: 5): ") or "5")

        if not output_filename.endswith('.mp4'):
            output_filename += '.mp4'

        processor = YOLOVideoProcessor(model_path)
        success = processor.process_video(
            youtube_url=youtube_url,
            output_filename=output_filename,
            start_time=start_time,
            end_time=end_time,
            max_duration=10,
            skip_frames=skip_frames
        )

        if success:
            processor.save_stats()
            print("\n🎉 처리가 성공적으로 완료되었습니다!")
        else:
            print("\n❌ 처리 실패!")

    except Exception as e:
        logger.error(f"❌ 오류 발생: {e}")


if __name__ == "__main__":
    main()


🎥 YouTube YOLO 영상 처리기
📋 YouTube 영상 URL을 입력하세요: https://youtu.be/JHva65QGlzE?si=fo4yuZFmiemoB57g

🤖 사용 가능한 YOLOv11 모델:
1. yolo11n.pt - YOLOv11 Nano - 가장 빠르고 가벼운 모델
2. yolo11s.pt - YOLOv11 Small - 속도와 정확도의 균형
3. yolo11m.pt - YOLOv11 Medium - 중간 크기 모델
4. yolo11l.pt - YOLOv11 Large - 높은 정확도
5. yolo11x.pt - YOLOv11 Extra Large - 최고 정확도

모델을 선택하세요 (1-5, 기본값: 1): 1

📹 제목: 서울의 홍대 홍익대 입구 근처의 최근 모습을 감상하세요 Travel destinations in Korea 韓国の目的地 4K
⏱ 길이: 177초 (02:57)
👤 업로더: Seoul_driver

⏰ 처리할 구간을 선택하세요 (0~177초)
🎬 시작 시간 (초 또는 분:초, 기본값: 0): 0
🏁 종료 시간 (1.0초~177초, 기본값: 자동): 30
💾 출력 파일명 (기본값: yolo_output.mp4): yolo11_qlry
⏭ 건너뛸 프레임 수 (기본값: 5): 5
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.35M/5.35M [00:00<00:00, 177MB/s]



🎉 처리가 성공적으로 완료되었습니다!


In [5]:
# 먼저 필요한 패키지들을 설치하세요
# !pip install ultralytics yt-dlp opencv-python matplotlib

import cv2
import yt_dlp
import tempfile
import os
import time
import matplotlib.pyplot as plt
import logging
from pathlib import Path
from typing import Optional, Tuple, List, Dict, Any
import argparse
import json
import sys
from datetime import datetime

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Jupyter 환경 감지
try:
    from IPython.display import clear_output
    JUPYTER_ENV = True
except ImportError:
    JUPYTER_ENV = False
    def clear_output(wait=True):
        pass

# 전역 모델 변수
model = None

class YOLOVideoProcessor:
    """
    향상된 YouTube 영상 처리기 with YOLOv11 객체 탐지
    """

    def __init__(self, model_path: str = "yolo11n.pt"):
        """
        처리기 초기화

        Args:
            model_path: YOLO 모델 경로
        """
        self.model_path = model_path
        self.model = None
        self.stats = {
            'total_frames_processed': 0,
            'total_detections': 0,
            'processing_time': 0,
            'detected_classes': set(),
            'input_fps': 0.0,
            'output_fps': 0.0,
            'fps_ratio': 0.0,
            'real_time_processing_fps': 0.0,
            'inference_fps': 0.0,
            'frame_skip_ratio': 0
        }

    def load_model(self) -> bool:
        """YOLOv11 모델 로드"""
        try:
            from ultralytics import YOLO
            self.model = YOLO(self.model_path)
            logger.info(f"🤖 YOLOv11 모델 로드 완료: {self.model_path}")
            return True
        except Exception as e:
            logger.error(f"❌ YOLOv11 모델 로드 실패: {e}")
            logger.error("💡 ultralytics 설치 필요: pip install ultralytics>=8.0.0")
            return False

    def get_video_info(self, youtube_url: str) -> Optional[Dict[str, Any]]:
        """
        YouTube 영상 정보 가져오기

        Args:
            youtube_url: YouTube 영상 URL

        Returns:
            영상 정보 딕셔너리 또는 실패시 None
        """
        ydl_opts = {
            'quiet': True,
            'no_warnings': True,
        }

        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info = ydl.extract_info(youtube_url, download=False)
                return {
                    'title': info.get('title', '알 수 없음'),
                    'duration': info.get('duration', 0),
                    'uploader': info.get('uploader', '알 수 없음'),
                    'view_count': info.get('view_count', 0),
                    'upload_date': info.get('upload_date', '알 수 없음'),
                    'description': info.get('description', '')[:200] + '...' if info.get('description') else '',
                    'url': youtube_url
                }
        except Exception as e:
            logger.error(f"❌ 영상 정보 가져오기 실패: {e}")
            return None

    def validate_time_range(self, start_time: float, end_time: float, duration: float) -> Tuple[float, float]:
        """
        시간 범위 검증 및 조정

        Args:
            start_time: 시작 시간(초)
            end_time: 종료 시간(초)
            duration: 영상 길이(초)

        Returns:
            검증된 (start_time, end_time) 튜플
        """
        start_time = max(0, start_time)
        end_time = min(end_time, duration)

        if start_time >= end_time:
            logger.warning(f"잘못된 시간 범위: {start_time}-{end_time}, 마지막 10초 사용")
            end_time = duration
            start_time = max(0, duration - 10)

        return start_time, end_time

    def format_time(self, seconds: float) -> str:
        """초를 MM:SS 형식으로 변환"""
        minutes = int(seconds // 60)
        seconds = int(seconds % 60)
        return f"{minutes:02d}:{seconds:02d}"

    def process_video(self, youtube_url: str, output_filename: str = "yolo_output.mp4",
                     start_time: float = 0, end_time: Optional[float] = None,
                     max_duration: int = 10, skip_frames: int = 5) -> bool:
        """
        YouTube 영상을 YOLO 탐지로 처리

        Args:
            youtube_url: YouTube 영상 URL
            output_filename: 출력 파일명
            start_time: 시작 시간(초)
            end_time: 종료 시간(초) (None이면 자동)
            max_duration: 최대 처리 길이
            skip_frames: 건너뛸 프레임 수

        Returns:
            성공시 True, 실패시 False
        """
        if not self.model and not self.load_model():
            return False

        # 출력 디렉토리 생성
        output_path = Path(output_filename)
        output_path.parent.mkdir(parents=True, exist_ok=True)

        # 다운로드 설정
        ydl_opts = {
            'format': 'mp4/best[height<=720]',  # 품질 제한 증가
            'outtmpl': os.path.join(tempfile.gettempdir(), 'temp_video.%(ext)s'),
            'quiet': True,
        }

        temp_video_path = None
        cap = None
        out = None

        try:
            # 영상 다운로드
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                logger.info("⬇️ YouTube 영상 다운로드 중...")
                info = ydl.extract_info(youtube_url, download=True)
                temp_video_path = ydl.prepare_filename(info)
                logger.info(f"🎥 제목: {info['title']}")
                logger.info(f"📊 길이: {info.get('duration', '알 수 없음')}초")

            # 비디오 캡처 초기화
            cap = cv2.VideoCapture(temp_video_path)
            if not cap.isOpened():
                raise ValueError("영상 파일을 열 수 없습니다")

            # 영상 속성 가져오기
            fps = cap.get(cv2.CAP_PROP_FPS)
            frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            video_duration = total_frames / fps

            # FPS 정보 저장
            self.stats['input_fps'] = fps
            self.stats['output_fps'] = fps / skip_frames
            self.stats['fps_ratio'] = self.stats['output_fps'] / fps
            self.stats['frame_skip_ratio'] = skip_frames

            # 시간 범위 설정
            if end_time is None:
                end_time = min(start_time + max_duration, video_duration)

            start_time, end_time = self.validate_time_range(start_time, end_time, video_duration)

            # 프레임 번호로 변환
            start_frame = int(start_time * fps)
            end_frame = int(end_time * fps)
            process_duration = end_time - start_time

            logger.info(f"📹 영상 정보: {frame_width}x{frame_height}")
            logger.info(f"🎬 원본 영상 FPS: {fps:.2f}")
            logger.info(f"🎯 출력 영상 FPS: {fps/skip_frames:.2f}")
            logger.info(f"⚡ 프레임 건너뛰기: {skip_frames}개마다 1개 처리 (처리율: {100/skip_frames:.1f}%)")
            logger.info(f"🎯 처리 구간: {self.format_time(start_time)} ~ {self.format_time(end_time)} ({process_duration:.1f}초)")

            # 시작 프레임으로 이동
            if start_frame > 0:
                cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

            # 비디오 라이터 초기화
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            output_fps = fps / skip_frames
            out = cv2.VideoWriter(output_filename, fourcc, output_fps, (frame_width, frame_height))

            if not out.isOpened():
                raise ValueError("출력 영상 파일을 생성할 수 없습니다")

            # 프레임 처리
            frame_num = start_frame
            processed_frames = 0
            inference_times = []
            start_processing_time = time.time()

            logger.info("🔄 YOLO 추론 시작...")

            while True:
                ret, frame = cap.read()
                if not ret or frame_num >= end_frame:
                    break

                # 성능을 위한 프레임 건너뛰기
                if (frame_num - start_frame) % skip_frames == 0:
                    try:
                        # YOLO 추론 시간 측정
                        inference_start = time.time()

                        # YOLO 추론 (기본 신뢰도 0.25 사용)
                        results = self.model.predict(
                            frame,
                            imgsz=640,
                            verbose=False,
                            show=False,
                            save=False
                        )[0]

                        inference_end = time.time()
                        inference_time = inference_end - inference_start
                        inference_times.append(inference_time)

                        # 결과 시각화
                        annotated_frame = results.plot()
                        out.write(annotated_frame)
                        processed_frames += 1

                        # 통계 업데이트
                        if results.boxes is not None:
                            detections = len(results.boxes)
                            self.stats['total_detections'] += detections

                            # 탐지된 클래스 추적
                            for box in results.boxes:
                                class_id = int(box.cls[0])
                                class_name = self.model.names[class_id]
                                self.stats['detected_classes'].add(class_name)

                        # 진행률 표시 (FPS 정보 포함)
                        if processed_frames % 30 == 0:
                            progress = ((frame_num - start_frame) / (end_frame - start_frame)) * 100
                            current_time = frame_num / fps
                            elapsed_time = time.time() - start_processing_time
                            processing_speed = processed_frames / elapsed_time if elapsed_time > 0 else 0

                            # 평균 추론 FPS 계산
                            avg_inference_time = sum(inference_times[-30:]) / len(inference_times[-30:]) if inference_times else 0
                            inference_fps = 1 / avg_inference_time if avg_inference_time > 0 else 0

                            logger.info(f"⏳ 진행률: {progress:.1f}% | 현재 시간: {current_time:.1f}초")
                            logger.info(f"📊 처리된 프레임: {processed_frames} | 실시간 처리속도: {processing_speed:.1f} FPS")
                            logger.info(f"🔥 YOLO 추론 속도: {inference_fps:.1f} FPS | 평균 추론시간: {avg_inference_time*1000:.1f}ms")

                    except Exception as e:
                        logger.warning(f"⚠️ 프레임 {frame_num} 처리 중 오류: {e}")
                        continue

                frame_num += 1

            # 최종 통계 계산
            total_time = time.time() - start_processing_time
            self.stats['processing_time'] = total_time
            self.stats['total_frames_processed'] = processed_frames
            self.stats['real_time_processing_fps'] = processed_frames / total_time if total_time > 0 else 0

            # 추론 FPS 계산
            if inference_times:
                avg_inference_time = sum(inference_times) / len(inference_times)
                self.stats['inference_fps'] = 1 / avg_inference_time if avg_inference_time > 0 else 0
            else:
                self.stats['inference_fps'] = 0

            processing_speed = processed_frames / total_time if total_time > 0 else 0
            playback_speed = process_duration / total_time if total_time > 0 else 0

            # 상세한 FPS 정보 출력
            logger.info(f"✅ 영상 추론 완료!")
            logger.info(f"📊 처리 구간: {start_time:.1f}초 ~ {end_time:.1f}초 ({process_duration:.1f}초)")
            logger.info(f"📊 총 처리 시간: {total_time:.2f}초")
            logger.info(f"📊 처리된 프레임: {processed_frames}")
            logger.info("")
            logger.info(f"🎬 === FPS 상세 정보 ===")
            logger.info(f"🎥 원본 영상 FPS: {self.stats['input_fps']:.2f}")
            logger.info(f"🎯 출력 영상 FPS: {self.stats['output_fps']:.2f}")
            logger.info(f"⚡ 실시간 처리 속도: {self.stats['real_time_processing_fps']:.2f} FPS")
            logger.info(f"🔥 YOLO 추론 속도: {self.stats['inference_fps']:.2f} FPS")
            logger.info(f"🔄 처리 배속: {playback_speed:.2f}x (1.0x = 실시간)")
            logger.info(f"📈 FPS 비율: {self.stats['fps_ratio']:.3f} (출력/입력)")
            logger.info(f"⏭ 프레임 건너뛰기: {self.stats['frame_skip_ratio']}개마다 1개 처리")
            logger.info("")

            if self.stats['total_detections'] > 0:
                logger.info(f"📊 총 탐지 수: {self.stats['total_detections']}")
                logger.info(f"📊 탐지된 클래스: {', '.join(sorted(self.stats['detected_classes']))}")
            logger.info(f"💾 저장된 파일: {output_filename}")

            # 파일 크기 확인
            if os.path.exists(output_filename):
                file_size = os.path.getsize(output_filename) / (1024*1024)
                logger.info(f"📁 파일 크기: {file_size:.2f} MB")

            return True

        except Exception as e:
            logger.error(f"❌ 오류 발생: {e}")
            return False

        finally:
            # 리소스 정리
            if cap:
                cap.release()
            if out:
                out.release()
            if temp_video_path and os.path.exists(temp_video_path):
                try:
                    os.remove(temp_video_path)
                    logger.info("🗑️ 임시 파일 정리 완료")
                except:
                    pass

    def save_stats(self, filename: str = "processing_stats.json"):
        """처리 통계를 파일에 저장"""
        stats_copy = self.stats.copy()
        stats_copy['detected_classes'] = list(stats_copy['detected_classes'])
        stats_copy['timestamp'] = datetime.now().isoformat()

        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(stats_copy, f, indent=2, ensure_ascii=False)

        logger.info(f"📊 통계가 {filename}에 저장되었습니다")

    def print_fps_summary(self):
        """FPS 요약 정보 출력"""
        print("\n" + "="*50)
        print("📊 FPS 성능 요약")
        print("="*50)
        print(f"🎥 원본 영상 FPS: {self.stats['input_fps']:.2f}")
        print(f"🎯 출력 영상 FPS: {self.stats['output_fps']:.2f}")
        print(f"⚡ 실시간 처리 속도: {self.stats['real_time_processing_fps']:.2f} FPS")
        print(f"🔥 YOLO 추론 속도: {self.stats['inference_fps']:.2f} FPS")
        print(f"📈 FPS 비율 (출력/입력): {self.stats['fps_ratio']:.3f}")
        print(f"⏭ 프레임 건너뛰기: {self.stats['frame_skip_ratio']}개마다 1개 처리")
        print(f"📊 총 처리 프레임: {self.stats['total_frames_processed']}")
        print(f"⏱ 총 처리 시간: {self.stats['processing_time']:.2f}초")
        print("="*50)


def get_available_models() -> Dict[str, str]:
    """사용 가능한 YOLOv11 모델 목록"""
    return {
        'yolo11n.pt': 'YOLOv11 Nano - 가장 빠르고 가벼운 모델',
        'yolo11s.pt': 'YOLOv11 Small - 속도와 정확도의 균형',
        'yolo11m.pt': 'YOLOv11 Medium - 중간 크기 모델',
        'yolo11l.pt': 'YOLOv11 Large - 높은 정확도',
        'yolo11x.pt': 'YOLOv11 Extra Large - 최고 정확도'
    }


def interactive_setup() -> Tuple[str, str, float, Optional[float]]:
    """대화형 설정 인터페이스"""
    print("\n🎥 YouTube YOLO 영상 처리기")
    print("=" * 50)

    # YouTube URL 입력
    youtube_url = input("📋 YouTube 영상 URL을 입력하세요: ").strip()
    if not youtube_url:
        raise ValueError("URL이 필요합니다")

    # 모델 선택
    models = get_available_models()
    print("\n🤖 사용 가능한 YOLOv11 모델:")
    for i, (model_name, description) in enumerate(models.items(), 1):
        print(f"{i}. {model_name} - {description}")

    while True:
        try:
            choice = input(f"\n모델을 선택하세요 (1-{len(models)}, 기본값: 1): ").strip()
            if not choice:
                choice = "1"
            choice = int(choice)
            if 1 <= choice <= len(models):
                model_path = list(models.keys())[choice-1]
                break
            else:
                print(f"❌ 1부터 {len(models)} 사이의 숫자를 입력하세요")
        except ValueError:
            print("❌ 올바른 숫자를 입력하세요")

    # 영상 정보 가져오기
    processor = YOLOVideoProcessor()
    video_info = processor.get_video_info(youtube_url)

    if not video_info:
        raise ValueError("영상 정보를 가져올 수 없습니다")

    duration = video_info['duration']
    print(f"\n📹 제목: {video_info['title']}")
    print(f"⏱ 길이: {duration}초 ({processor.format_time(duration)})")
    print(f"👤 업로더: {video_info['uploader']}")

    # 시간 범위 선택
    print(f"\n⏰ 처리할 구간을 선택하세요 (0~{duration}초)")
    start_time = float(input(f"🎬 시작 시간 (초 또는 분:초, 기본값: 0): ") or "0")

    # 시간 형식 처리 (MM:SS)
    if isinstance(start_time, str) and ':' in str(start_time):
        parts = str(start_time).split(':')
        if len(parts) == 2:
            start_time = int(parts[0]) * 60 + int(parts[1])

    end_input = input(f"🏁 종료 시간 ({start_time+1}초~{duration}초, 기본값: 자동): ").strip()
    end_time = None
    if end_input:
        if ':' in end_input:
            parts = end_input.split(':')
            if len(parts) == 2:
                end_time = int(parts[0]) * 60 + int(parts[1])
        else:
            end_time = float(end_input)

    return youtube_url, model_path, start_time, end_time


def main():
    """메인 함수 with 명령줄 인터페이스"""
    # Jupyter 환경 체크 및 인수 정리
    if JUPYTER_ENV or any('-f' in arg for arg in sys.argv):
        # Jupyter 환경에서는 인수를 초기화
        sys.argv = [sys.argv[0]]

    parser = argparse.ArgumentParser(description='YouTube 영상 처리기 with YOLOv11')
    parser.add_argument('--url', type=str, help='YouTube 영상 URL')
    parser.add_argument('--model', type=str, default='yolo11n.pt', help='YOLO 모델 경로')
    parser.add_argument('--start', type=float, default=0, help='시작 시간(초)')
    parser.add_argument('--end', type=float, help='종료 시간(초)')
    parser.add_argument('--duration', type=int, default=10, help='최대 처리 길이(초)')
    parser.add_argument('--output', type=str, default='yolo_output.mp4', help='출력 파일명')
    parser.add_argument('--skip-frames', type=int, default=5, help='건너뛸 프레임 수')
    parser.add_argument('--interactive', action='store_true', help='대화형 모드 사용')

    # Jupyter 환경에서도 알 수 없는 인수 허용
    parser.add_argument('-f', '--connection-file', help='Jupyter kernel connection file (ignored)')

    try:
        args = parser.parse_args()
    except SystemExit:
        # argparse 오류 발생 시 대화형 모드로 전환
        args = argparse.Namespace(
            url=None, model='yolo11n.pt', start=0, end=None,
            duration=10, output='yolo_output.mp4', skip_frames=5,
            interactive=True
        )

    try:
        if args.interactive or not args.url:
            # 대화형 모드
            youtube_url, model_path, start_time, end_time = interactive_setup()
            output_filename = input("💾 출력 파일명 (기본값: yolo_output.mp4): ") or "yolo_output.mp4"
            skip_frames = int(input("⏭ 건너뛸 프레임 수 (기본값: 5): ") or "5")
        else:
            # 명령줄 모드
            youtube_url = args.url
            model_path = args.model
            start_time = args.start
            end_time = args.end
            output_filename = args.output
            skip_frames = args.skip_frames

        # 출력 파일 확장자 확인
        if not output_filename.endswith('.mp4'):
            output_filename += '.mp4'

        # 영상 처리
        processor = YOLOVideoProcessor(model_path)
        success = processor.process_video(
            youtube_url=youtube_url,
            output_filename=output_filename,
            start_time=start_time,
            end_time=end_time,
            max_duration=args.duration,
            skip_frames=skip_frames
        )

        if success:
            # 통계 저장
            processor.save_stats()
            processor.print_fps_summary()  # FPS 요약 출력
            print("\n🎉 처리가 성공적으로 완료되었습니다!")
            print("👋 프로그램 종료")
        else:
            print("\n❌ 처리 실패!")
            print("👋 프로그램 종료")

    except KeyboardInterrupt:
        print("\n🚫 사용자에 의해 중단되었습니다")
    except Exception as e:
        logger.error(f"❌ 예상치 못한 오류: {e}")


# 빠른 실행을 위한 함수
def quick_process(youtube_url: str, start_time: float = 0, duration: int = 10):
    """빠른 처리를 위한 간단한 함수"""
    processor = YOLOVideoProcessor("yolo11n.pt")  # 가장 빠른 모델 사용
    success = processor.process_video(
        youtube_url=youtube_url,
        start_time=start_time,
        max_duration=duration,
        skip_frames=3  # 더 빠른 처리를 위해 프레임 수 조정
    )
    if success:
        processor.print_fps_summary()
    return success


# Jupyter/Colab에서 직접 실행하기 위한 함수
def run_in_jupyter():
    """Jupyter/Colab에서 직접 실행"""
    try:
        youtube_url, model_path, start_time, end_time = interactive_setup()
        output_filename = input("💾 출력 파일명 (기본값: yolo_output.mp4): ") or "yolo_output.mp4"
        skip_frames = int(input("⏭ 건너뛸 프레임 수 (기본값: 5): ") or "5")

        if not output_filename.endswith('.mp4'):
            output_filename += '.mp4'

        processor = YOLOVideoProcessor(model_path)
        success = processor.process_video(
            youtube_url=youtube_url,
            output_filename=output_filename,
            start_time=start_time,
            end_time=end_time,
            max_duration=10,
            skip_frames=skip_frames
        )

        if success:
            processor.save_stats()
            processor.print_fps_summary()
            print("\n🎉 처리가 성공적으로 완료되었습니다!")
            print("👋 프로그램 종료")
        else:
            print("\n❌ 처리 실패!")
            print("👋 프로그램 종료")

    except Exception as e:
        logger.error(f"❌ 오류 발생: {e}")


if __name__ == "__main__":
    main()


🎥 YouTube YOLO 영상 처리기
📋 YouTube 영상 URL을 입력하세요: https://youtu.be/JHva65QGlzE?si=fo4yuZFmiemoB57g

🤖 사용 가능한 YOLOv11 모델:
1. yolo11n.pt - YOLOv11 Nano - 가장 빠르고 가벼운 모델
2. yolo11s.pt - YOLOv11 Small - 속도와 정확도의 균형
3. yolo11m.pt - YOLOv11 Medium - 중간 크기 모델
4. yolo11l.pt - YOLOv11 Large - 높은 정확도
5. yolo11x.pt - YOLOv11 Extra Large - 최고 정확도

모델을 선택하세요 (1-5, 기본값: 1): 1

📹 제목: 서울의 홍대 홍익대 입구 근처의 최근 모습을 감상하세요 Travel destinations in Korea 韓国の目的地 4K
⏱ 길이: 177초 (02:57)
👤 업로더: Seoul_driver

⏰ 처리할 구간을 선택하세요 (0~177초)
🎬 시작 시간 (초 또는 분:초, 기본값: 0): 0
🏁 종료 시간 (1.0초~177초, 기본값: 자동): 30
💾 출력 파일명 (기본값: yolo_output.mp4): yolo11_fps
⏭ 건너뛸 프레임 수 (기본값: 5): 5

📊 FPS 성능 요약
🎥 원본 영상 FPS: 29.97
🎯 출력 영상 FPS: 5.99
⚡ 실시간 처리 속도: 46.78 FPS
🔥 YOLO 추론 속도: 80.39 FPS
📈 FPS 비율 (출력/입력): 0.200
⏭ 프레임 건너뛰기: 5개마다 1개 처리
📊 총 처리 프레임: 180
⏱ 총 처리 시간: 3.85초

🎉 처리가 성공적으로 완료되었습니다!
👋 프로그램 종료
